<h3 style="text-align: center;"><b>FashionMNIST (5 баллов)</b></h3>

Для обучения нейросетей на TensorFlow обычно используют либо модуль `.keras` (который, по сути, является Keras'ом, только обращение к нему происходит через TF), либо модуль `tf.Slim`. В этом задании Вам нужно самостоятельно написть обучение нейросети на датасете FashionMNIST и добиться качества (поклассового `accuracy`) **больше, чем 0.9**.

Бейзланом к этому заданию является официальный туториал TensorFlow: https://www.tensorflow.org/tutorials/keras/basic_classification

*Hint*: можно использовать свёрточные слои и слои не из туториала

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist

train_data, test_data = fashion_mnist.load_data()



train_data = list(train_data)
train_data[0] = train_data[0]
train_data[1] = train_data[1]

test_data = list(test_data)

In [3]:
std = train_data[0].std()
mean = train_data[0].mean()


train_data[0] = (train_data[0] - mean) / std
test_data[0] = (test_data[0] - mean) / std

In [4]:
image = tf.placeholder(tf.float32, shape=[None, 28, 28])
labels = tf.placeholder(tf.int64, shape=[None])
labels_one_hot = tf.one_hot(labels, depth=10)
# 086 sigm
expanded = tf.expand_dims(image, 3)
conv1 = tf.layers.conv2d(expanded, 128, (3, 3))
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
conv2 = tf.layers.conv2d(pool1, 64, (3, 3))
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

flattened = tf.layers.flatten(pool2)
print(flattened)
l1 = tf.layers.dense(flattened, units=128, activation=tf.nn.sigmoid)
l2 = tf.layers.dense(l1, units=64, activation=tf.nn.sigmoid)
output = tf.layers.dense(l2, units=10, activation='softmax')

loss = tf.losses.softmax_cross_entropy(labels_one_hot, output)
accuracy, accuracy_op = tf.metrics.accuracy(labels=tf.argmax(labels_one_hot, 1), 
                                            predictions=tf.argmax(output, 1))

my_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(output, 1), labels), tf.float32))

optimizer = tf.train.RMSPropOptimizer(0.001)
train = optimizer.minimize(loss)

W0709 15:27:54.914943 13688 deprecation.py:323] From <ipython-input-4-eeea536efc3f>:6: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0709 15:27:54.918974 13688 deprecation.py:506] From C:\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0709 15:27:55.113267 13688 deprecation.py:323] From <ipython-input-4-eeea536efc3f>:7: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0709 15:27:55.242045 13688 deprecation.py:323] From <ipython-input-4-eeea536efc3f>:11: flatten (fr

Tensor("flatten/Reshape:0", shape=(?, 1600), dtype=float32)


W0709 15:27:55.897379 13688 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow\python\ops\losses\losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0709 15:27:55.985264 13688 deprecation.py:506] From C:\Anaconda3\lib\site-packages\tensorflow\python\training\rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
tf.trainable_variables()

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 128) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 128, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(1600, 128) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(128, 64) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(64, 10) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(10,) dtype=float32_ref>]

In [6]:
def dataset_generator(dataset, batch_size=128):
    i = 0
    while True:
        yield {
                'data': dataset[0][i * batch_size:(i + 1) * batch_size],
                'labels': dataset[1][i * batch_size:(i + 1) * batch_size] 
              }
        i += 1
        if i * batch_size >= len(dataset[0]):
            i = 0

In [ ]:
from tqdm import tqdm_notebook
with tf.device('/gpu:0'):
    sess = tf.Session()
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())

    writer = tf.summary.FileWriter('.', sess.graph)

    train_generator = dataset_generator(train_data)
    test_generator = dataset_generator(test_data)
    history = {'loss': [],
               'val_loss': [],
               'accuracy': [],
               'val_accuracy': []}

    for i in tqdm_notebook(range(15000)):
        train_batch = next(train_generator)
        test_batch = next(test_generator)

        _, train_accuracy, train_loss = sess.run((train, my_accuracy, loss), 
                                             feed_dict={image: train_batch['data'], 
                                                        labels: train_batch['labels']})


        val_accuracy, val_loss = sess.run((my_accuracy, loss), feed_dict={image: test_batch['data'], 
                                                                          labels: test_batch['labels']})

        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['accuracy'].append(train_accuracy)
        history['val_accuracy'].append(val_accuracy)

In [ ]:
def smoothen_curve(points, factor=51):
    from scipy.signal import savgol_filter
    return savgol_filter(points, factor, 3) 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(np.arange(len(history['loss'])), smoothen_curve(history['loss']), label='train')
ax1.plot(np.arange(len(history['val_loss'])), smoothen_curve(history['val_loss']), label='test')
ax1.legend()
ax1.set_title('Loss')

ax2.plot(np.arange(len(history['accuracy'])), smoothen_curve(history['accuracy']), label='train')
ax2.plot(np.arange(len(history['val_accuracy'])), smoothen_curve(history['val_accuracy']), label='test')
ax2.set_title('Accuracy')
ax2.legend()

In [ ]:
validation_generator = dataset_generator(test_data, 1)
s = 0
for i in range(len(test_data[0])):
    batch = next(validation_generator)
    s += sess.run(my_accuracy, feed_dict={
            image: batch['data'],
            labels: batch['labels']
        })
print(s / len(test_data[0]))

#### CIFAR10:
Бейзлайн по классификации CIFAR10: https://www.tensorflow.org/tutorials/images/deep_cnn  
Там есть ссылки на файлы с кодом, который тоже можно прямо сюда вставлять, главное, чтобы качество в итоге получилось хорошим (формально порог не задан, будет смотреться у всех индивиуально).

In [ ]:
<Покажите класс>

In [ ]:
...